In [1]:
%load_ext autoreload
%autoreload 2
from sklearn import set_config; set_config(display='diagram')

In [2]:
# import the dataset from s3 bucket 
import pandas as pd
url = "s3://wagon-public-datasets/taxi-fare-train.csv"

# Select only 10 000 rows while creating the DataFrame
df = pd.read_csv(url, nrows=10_000)

X=df.drop(columns=['fare_amount'])
y=df['fare_amount']

In [5]:
from TaxiFareModel.trainer import Trainer

In [6]:
trainer = Trainer(X,y)

In [7]:
trainer.set_pipeline()

Pipeline(steps=[('feat_eng',
                 ColumnTransformer(transformers=[('time',
                                                  Pipeline(steps=[('timefeaturesencoder',
                                                                   TimeFeaturesEncoder(time_column='pickup_datetime')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['pickup_datetime']),
                                                 ('distance',
                                                  Pipeline(steps=[('distancetransformer',
                                                                   DistanceTransformer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['pickup_latitude',
                                                   'pickup_longitude',
                                                   'dropoff_latitude',
                                                   'dropoff_longitude'])])),
                ('regressor', RandomForestRegressor())])

In [9]:
from sklearn.model_selection import train_test_split
from TaxiFareModel.data import get_data, clean_data
from TaxiFareModel.trainer import Trainer

N = 10_000
df = get_data(nrows=N)
df = clean_data(df)
y = df["fare_amount"]
X = df.drop("fare_amount", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
trainer = Trainer(X_train, y_train)
trainer.run()
trainer.evaluate(X_test, y_test)

ValueError: Found unknown categories [2008] in column 3 during transform